1 - OBTENER ACTIVOS A ANALIZAR 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandas_datareader import data as web
import warnings
import yfinance as yfin
yfin.pdr_override()
warnings.filterwarnings("ignore")
from csv import reader

            #Ejemplo de la información a obtener de un indice
            #df = web.get_data_yahoo("TSLA", start="2020-01-01", end="2023-01-01")
            #print(df)
            #                      Open        High         Low       Close   Adj Close     Volume
            #    Date
            #    2020-01-02   28.299999   28.713333   28.114000   28.684000   28.684000  142981500
            #    2020-01-03   29.366667   30.266666   29.128000   29.534000   29.534000  266677500
            #    2020-01-06   29.364668   30.104000   29.333332   30.102667   30.102667  151995000
            #    2020-01-07   30.760000   31.441999   30.224001   31.270666   31.270666  268231500
            #    2020-01-08   31.580000   33.232666   31.215334   32.809334   32.809334  467164500
            #    ...                ...         ...         ...         ...         ...        ...

prices = pd.DataFrame() #Crear dataframe de precios

                    #Crear lista de indices manualmente
tickers = ['ITUB3.SA', 'BBDC3.SA', 'BBAS3.SA', 'SANB3.SA', '^BVSP']

#Leer de un archivo CSV el listado de indices a analizar.
# with open('C:/Users/alvar/OneDrive/Escritorio/IndicesCSV.csv', 'r') as archivo:
#     leer = reader(archivo)
#     lista2D = list(leer)

# lista2D.pop(0) #Elimina primer elemento de la lista
        
# #Declaramos tickers
# tickers = [] 
# for item in lista2D:
#     #Añadir elementos a la lista 1D
#     tickers += item
# # Visulaizar resultados de la lista plana o 1D
# print(tickers)

2 - OBTENER DATOS DE PRECIOS

In [ ]:
for i in tickers:
    prices[i] = web.get_data_yahoo(i,'2019-01-01')['Adj Close']

                    #Primero, importe las cotizaciones de cierre ajustadas para 
                    # ITUB3, BBAS3, BBDC3, SANB3 y el Índice Bovespa ^BVSP. 
                    # El ticker correcto que se utilizará en el código se puede consultar fácilmente en Yahoo Finance.
                    # Así que lo que los datos de cotizaciones ajustados por splits y dividendos (Adj Close) 
                    # del 01/01/2008 al 15/03/2020.
                    # Para una mejor visualización, se renombró las columnas y se transformó el puntaje del Ibovespa, 
                    # dividiendo sus valores por 1000.

prices['^BVSP'] = prices['^BVSP']/1000

#-------------
#Esta linea resetea el indice actual que sería Date
prices.reset_index(inplace = True)
#-------------

prices.dropna(subset = ['^BVSP'], inplace = True)

#En principio no hay problema si el nombre de los indices a valorar contienen "." o "^"
prices.rename(columns ={'ITUB3.SA':'ITUB', 'BBDC3.SA':'BBDC','BBAS3.SA':'BBAS','SANB3.SA':'SANB', '^BVSP':'IBOV'},inplace = True)

#--------------------------
#Esta linea es muy útil pues establece que la lista tickers contenga el valor del df.prices 
#coguiendo los nombres de sus indices que ya estan cambiados | axis = 0 fila | axis = 1 columna | .columns(solo columna)
tickers = list(prices.drop(['Date'], axis = 1).columns)
#--------------------------

#######################-------------------

# USO DE SET_INDEX PARA MODIFICAR INDICES DE UN DATAFRAME
# df.set_index(keys, drop=True, append=False, inplace=False, verify_integrity=False)
# en donde las propiedades más importantes son:

# keys: los nombres de las columnas, pudiendo ser una o varias, que se establecerán como índice del DataFrame.
# drop: indica si se eliminará la columna del conjunto de datos una vez establecida esta como índice, el valor por defecto es verdadero, por lo que las columnas se eliminarán.
# append: valor que se indica si la columna se agrega al índice ya existente, por defecto el valor es falso, por lo que el índice actual será reemplazado por la nueva columna.
# inplace: indica si el método aplica las modificaciones al propio objeto o devuelve una copia modificada del mismo, por defecto el valor es falso, por lo que se obtiene una copia modificada.

prices.set_index(keys='Date', drop=False, inplace=True)

########################----------------------


###############################
###  MOVER COLUMNAS DE UN DATAFRAME

# df = pd.DataFrame(data)

# shift column 'Name' to first position
# first_column = df.pop('Name')
  
# insert column using insert(position,column_name, first_column) function
# df.insert(0, 'Name', first_column)

###############################

3 - COTIZACION POR TIEMPO Y MEDIAS

In [ ]:
#COTIZACIÓN POR TIEMPO

# plt.figure(figsize=(16,6))

# for i in tickers:
#         plt.plot(prices['Date'], prices[i])
# plt.legend(tickers)
# plt.grid()
# plt.title("Cotización x tiempo", fontsize = 25)
# plt.show()

#Una forma muy interesante de analizar el comportamiento de un activo a largo plazo es visualizar
#sus medias móviles, y esto se puede hacer de forma muy sencilla con el paquete Pandas. 
#Utilizando el método .rolling de Pandas para crear y trazar promedios móviles trimestrales y anuales de ITUB3 como ejemplo.
#Modificando el parametro Window podemos modificar el cálculo de la media para hacer la que deseemos.


mm7 = pd.DataFrame()
mm14 = pd.DataFrame()
mm30 = pd.DataFrame()
mm90 = pd.DataFrame()
mm365 = pd.DataFrame()

plt.figure(figsize=(16,6))
for i in tickers:
    #Media Semanal
    plt.plot(prices['Date'],prices[i].rolling(window = 7).mean())
    mm7['Date'] = prices['Date']
    mm7[i]=prices[i].rolling(window = 7).mean() #Añade el valor al dataframe para media semanal del indice que se le pasa
    #Media Dos semanas
    plt.plot(prices['Date'],prices[i].rolling(window = 14).mean())
    mm14['Date'] = prices['Date']
    mm14[i]=prices[i].rolling(window = 14).mean() #Añade el valor al dataframe para media 14 días del indice que se le pasa
    #Media Mensual
    plt.plot(prices['Date'],prices[i].rolling(window = 30).mean())
    mm30['Date'] = prices['Date']
    mm30[i]=prices[i].rolling(window = 30).mean() #Añade el valor al dataframe para media mensual del indice que se le pasa
    #Media Trimestral
    plt.plot(prices['Date'],prices[i].rolling(window = 90).mean())
    mm90['Date'] = prices['Date']
    mm90[i]=prices[i].rolling(window = 90).mean() #Añade el valor al dataframe para media trimestral del indice que se le pasa
    #Cotización diaria
    plt.plot(prices['Date'], prices[i], alpha = 0.8)
    #Media Anual
    plt.plot(prices['Date'],prices[i].rolling(window = 365).mean())
    mm365['Date'] = prices['Date']
    mm365[i]=prices[i].rolling(window = 365).mean() #Añade el valor al dataframe para media trimestral del indice que se le pasa 

plt.grid()
plt.title('Cotizaciones diarias y médias móviles', fontsize = 15)
plt.legend(['Media semanal','Media 14 dias','Media movil mensual','Media movil trimestral','Cotización diaria','Media móvil anual'])
plt.show()

# Tracé un mapa de calor de la correlación de activos. prices.corr()
# me devuelve una matriz numérica de los valores de correlación de las cotizaciones,
# pero es mucho más agradable visualizar estos datos en un mapa de calor. 
# El parámetro annot = True hace que los valores sean visibles en el mapa.

#sns.heatmap(prices.corr(), annot = True)
#plt.show()

4 - RETORNO DIARIO

In [ ]:
#RETORNO DIARIO

#Con la cotización histórica, también podemos crear un nuevo DataFrame de retorno diario.
# El retorno es la variación relativa que se produce en cualquier tipo de activo y en particular, 
# en un activo financiero durante un período de tiempo determinado(En este caso diario). 
# Un inversionista puede ganar, perder o mantener su capital cuando compra un activo durante un período de tiempo definido. 
# utilizando el método pct_change(). 
# Con esto, fue posible obtener información más profunda sobre los activos 
# utilizando los paquetes Pandas, Seaborn y Matplotlib.

returns = pd.DataFrame()

for i in tickers:
    returns[i] = prices[i].pct_change()
returns['Date'] = prices['Date']

returns = returns.describe()

print(returns)

#Esta sencilla tabla tiene información muy rica, principalmente la desviación estándar
# y el promedio de rendimiento de las acciones. 
# La desviación estándar (std) del rendimiento diario representa la volatilidad. 
# El índice IBOV fue menos volátil que las acciones de los 4 bancos en este período,
# pero también tuvo un retorno diario promedio bajo (media = 0.000202).

5 - RENTABILIDAD ACUMULADA Y DISTRIBUCION NORMAL

In [ ]:


#Pairplot nos permite visualizar las relaciones entre cada variable en nuestro conjunto de datos:

#sns.pairplot(returns)
#plt.show()

#La distribución de las variaciones diarias del Ibovespa es una distribución normal:

sns.distplot(returns['IBOV'].dropna())

#RENTABILIDAD ACUMULADA

#El rendimiento acumulado de un activo se calcula multiplicando los rendimientos diarios 
# sumados por 1. Por ejemplo: el rendimiento acumulado durante 3 días cuando una acción subió 
# un 2 % es (1,02 x 1,02 x 1,02) = 1,0612. Hice este cálculo aplicando el método cumsum() 
# al DataFrame de las devoluciones diarias. Este gráfico muestra la rentabilidad acumulada 
# desde el 01/01/2008.

return_sum = pd.DataFrame()
for ticker in tickers:
    return_sum[ticker] = (returns[ticker]+1).cumprod()
return_sum['Date'] = returns['Date']

plt.figure(figsize=(16,6))
plt.plot(return_sum['Date'], return_sum.drop(['Date'], axis = 1), alpha = 0.9)
plt.legend(tickers)
plt.title("Retorno x tiempo", fontsize = 15)
plt.grid()
plt.show()
print(return_sum)
